In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import xmltodict

from pydantic import BaseModel
from typing import List, Dict

import metadata_update_pro
import validation_pro 

METADATA_FOLDER = "metadata_xml/"

Reminder to check out `uuidref`...if we keep lifting template, we would copy this part over.
```
default2 = default_template["ns0:MD_Metadata"]
(default2["ns0:identificationInfo"]
 ["ns0:MD_DataIdentification"]["ns0:descriptiveKeywords"][0]["ns0:MD_Keywords"]["ns0:thesaurusName"]
)
```

In [ ]:
# second run onward....having trouble key in
#https://stackoverflow.com/questions/58763774/cannot-parse-json-to-xml-using-xmltodict-unparse-when-converting-json-list
XML_FILE = "./ca_transit_routes.xml"
esri_metadata = metadata_update_pro.xml_to_json(XML_FILE)


### Now find all instances where what's inputted in BaseModel is used

In [ ]:
ROUTES_DICT = {
    "dataset_name": "ca_transit_routes", 
    "publish_entity": "California Integrated Travel Project", 

    "abstract": "Public. EPSG: 4326",
    "purpose": "my purpose", # purpose, also repeat for description?

    "beginning_date": "2022/09/14",
    "end_date": "2022-10-14",
    "place": "California",
    "edition": '2',
    
    "status": "completed", 
    "frequency": "monthly",
    
    "theme_keywords": ["1", "2", "3", "4", "5"], #tags

    "methodology": "this is my methodology", 
    "contact_organization": "Caltrans", 
    "contact_person": "Tiffany Chu", 
    "contact_email": "tiffany.chu@dot.ca.gov",
    
    "horiz_accuracy": "4 meters",
}


In [ ]:
updated_meta = metadata_update_pro.overwrite_metadata_json(esri2, DATASET_INFO_VALIDATED)

In [ ]:
updated_meta

In [1]:
import os

import metadata_update_pro
from gcs_to_esri import remove_zipped_shapefiles

# Import various dictionaries
import hqta
import traffic_ops

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
ROUTES_DICT2 = {
    "dataset_name": "ca_transit_routes", 
    "publish_entity": "California Integrated Travel Project", 

    "abstract": "Public. EPSG: 4326",
    "purpose": "my purpose", # purpose, also repeat for description?

    "beginning_date": "2022/09/14",
    "end_date": "2022-10-14",
    "place": "California",
    "edition": '2',
    
    "status": "completed", 
    "frequency": "monthly",
    
    "theme_keywords": ["1", "2", "3", "4", "5"], #tags

    "methodology": "this is my methodology", 
    "contact_organization": "Caltrans", 
    "contact_person": "Tiffany Chu", 
    "contact_email": "tiffany.chu@dot.ca.gov",
    
    "horiz_accuracy": "4 meters",
}


In [4]:
OPEN_DATA = {

    "transit_routes": {
        "path": "./metadata_xml/ca_transit_routes.xml",
        "metadata_dict": ROUTES_DICT2,
    },
}

RUN_ME = [
    "transit_routes",
]

for name, dataset in OPEN_DATA.items():
    if name in RUN_ME:
        print(name)
        print("-------------------------------------------")
        metadata_update_pro.update_metadata_xml(dataset["path"], 
                                            dataset["metadata_dict"], 
                                            first_run=False)

transit_routes
-------------------------------------------
Loading XML as JSON from ./metadata_xml/ca_transit_routes.xml
Read in XML as JSON
Skip default template.
Overwrite JSON using dict
Convert JSON back to XML
Save over existing XML
